In [ ]:
import os
import string
import pandas as pd

primitives_path = os.path.join(os.path.dirname("__file__"), "primitives")
processed_path = os.path.join(os.path.dirname("__file__"), "processed")

spotify_dataset_file = "Spotify 2010 - 2019 Top 100.csv"
lagos_dataset_file = "Lagos Argentina - Hoja 1.csv"
fifa_dataset_file = "FIFA-21 Complete.csv"


: 

In [ ]:
def importador_dataFrame_csv(dataset_name):
    """modulo que abre los datasets originales, con separacion por "," para el de
    jugadores de fifa y "," para el resto"""
    
    if dataset_name == fifa_dataset_file:
        datos= pd.read_csv(os.path.join(primitives_path, dataset_name), sep=';')
    else:
        datos= pd.read_csv(os.path.join(primitives_path, dataset_name), sep=',')
    return pd.DataFrame(datos)

: 

Procesamiento de dataset de canciones de Spotify

In [ ]:
df_spotify=importador_dataFrame_csv(spotify_dataset_file)
df_spoty_filtrado= df_spotify.dropna()          #elimino los nan
df_spoty_filtrado= df_spoty_filtrado[['top genre', 'artist type', 'year released', 'top year', 'bpm', 'artist']]        # elimino columnas que no me sirven

: 

In [ ]:
def cambiar_formato(genero):
    """Este procedimiento toma el campo top genre y a cada palabra le realiza una operacion dependiendo de cual es:
        -Si es k-pop, la cambia por K-Pop
        -Si es alguno de los generos 'edm','dfw','uk','r&b' o 'lgbtq+', los pone en mayusculas
        -Si no es ninguno de los anteriores, pone en mayuscula la primera letra"""

    generos_minus = ['edm','dfw','uk','r&b','lgbtq+']
    generos_sep = genero.split(" ")
    genero_resultado = []
    for genero in generos_sep:
        if genero in generos_minus:
            genero = genero.upper()
        elif genero == "k-pop":
            genero = "K-Pop"
        else:
            genero = genero.capitalize()
        genero_resultado.append(genero)
    return " ".join(genero_resultado)

: 

Reemplazo en la columa de genero con el formato pedido, pasando 'edm','dfw','uk','r&b','lgbtq+' a mayuscula, 'k-pop' a K-Pop y los campos numericos a enteros

In [ ]:
df_spoty_filtrado['top genre'] = df_spoty_filtrado['top genre'].apply(lambda x: cambiar_formato(x))

df_spoty_filtrado['year released'] = df_spoty_filtrado['year released'].astype(int)
df_spoty_filtrado['top year'] = df_spoty_filtrado['top year'].astype(int)
df_spoty_filtrado['bpm'] = df_spoty_filtrado['bpm'].astype(int)

: 

Procesamiento dataset de lagos de Argentina

In [ ]:
def conversor_coord_decimal(coord):
    """procedimiento que convierte el contenido de la columna coordenada del dataFrame lagos,
    que esta en formato str(en grados) y se transforma de decimale y posteriormente retorna a str"""
    
    puntos_cardinales = {'N':1, 'S':-1, 'E': 1, 'O':-1}
    lat = coord.split(" ")[0].replace("°", " ").replace("'", " ").replace('"'," ").split(" ")
    long = coord.split(" ")[1].replace("°", " ").replace("'", " ").replace('"'," ").split(" ")
    lat_pc = lat.pop(-1)
    long_pc = long.pop(-1)
    latitud = (int(lat[0]) + (int(lat[1])/60) + (int(lat[2])/3600)) * puntos_cardinales[lat_pc]
    longitud = (int(long[0]) + (int(long[1])/60) + (int(long[2])/3600)) * puntos_cardinales[long_pc]
    return str(latitud) + " " + str(longitud)


: 

In [ ]:

df_lagos = importador_dataFrame_csv(lagos_dataset_file)
df_lagos_filtrado = df_lagos.dropna()           #elimimo los nan
#guardo en la variable df_lagos_filtrado un nuevo dataFrame con las columas corespondientes a lo pedido 
df_lagos_filtrado = df_lagos[['Ubicación', 'Superficie (km²)', 'Profundidad máxima (m)', 'Profundidad media (m)', 'Coordenadas', 'Nombre']]

#aplico con .map la funcion conversor_coord_decimal a la columna coordenadas
df_lagos_filtrado['Coordenadas'] = df_lagos_filtrado['Coordenadas'].map(conversor_coord_decimal)

df_lagos_filtrado['Ubicación'] = df_lagos_filtrado['Ubicación'].str.replace(',','.').str.replace('"', " ")

: 

Procesamiento de dataset de jugadores de FIFA

In [ ]:
df_fifa=importador_dataFrame_csv(fifa_dataset_file)

df_fifa=df_fifa.dropna()            #elimimo los nan

#guardo en la variable df_lagos_filtraddo un nuevo dataFrame con las columas corespondientes a lo pedido 
df_fifa_filtrado= df_fifa[['team', 'nationality', 'position', 'age', 'potential', 'name']]
df_fifa_filtrado['team']=df_fifa_filtrado['team'].str.strip()# quita los espacios

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6088\2462142610.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fifa_filtrado['team']=df_fifa_filtrado['team'].str.strip()


: 

In [ ]:
def obtener_potencial(potencial):
    """procedimiento que devuelve la calidad del jugador en str segun sus puntos de potencial recibidos"""
    
    if  potencial < 60:
        return "Regular"
    elif potencial >= 60 and potencial <= 79:
        return "Bueno"
    elif potencial > 80 <= 90:
        return "Muy Bueno"
    else:
        return "Sobresaliente"

def posicion_en_espanol(posicion):
    """procedimiento que divide las posiciones del jugador es una lista,
    asigna a cada posicion su nombre correspondiente en español y
    devuelve un string con las posiciones"""
    
    lista_pos = posicion.split("|")
    posiciones = {
        "GK": "Arquero",
        "LB": "Lateral Izquierdo",
        "LCB": "Defensor Central Izquierdo",
        "CB": "Defensor Central",
        "RCB": "Defensor Central Derecho",
        "RB": "Lateral Derecho",
        "LWB": "Carrilero Izquierdo",
        "LDM": "Mediocampista Defensivo Izquierdo",
        "CDM": "Mediocampista Defensivo Central",
        "RDM": "Mediocampista Defensivo Derecho",
        "RWB": "Carrilero Derecho",
        "LM": "Mediocampista Izquierdo",
        "LCM": "Mediocampista Central Izquierdo",
        "CM": "Mediocampista Central",
        "RCM": "Mediocampista Central Derecho",
        "RM": "Mediocampista Derecho",
        "LAM": "Mediocampista Ofensivo Izquierdo",
        "CAM": "Mediocampista Ofensivo Central",
        "RAM": "Mediocampista Ofensivo Derecho",
        "LW": "Extremo Izquierdo",
        "ST": "Segundo Delantero",
        "RW": "Extremo Derecho",
        "LF": "Delantero Izquierdo",
        "CF": "Media Punta",
        "RF": "Delantero Derecho",
    }
    lista_pos_esp = []
    lista_pos_esp = list(map(lambda pos: posiciones[pos], lista_pos))
        
    return "|".join(lista_pos_esp)

: 

Se aplica a todos los elementos de la columna potential con un map la funcion obtener_potencial

In [ ]:
df_fifa_filtrado['potential']=df_fifa_filtrado['potential'].map(obtener_potencial)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6088\2295867570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fifa_filtrado['potential']=df_fifa_filtrado['potential'].map(obtener_potencial)


: 

Se aplica a todos los elementos de la columna potential con un map la funcion posicion_en_espanol

In [ ]:
df_fifa_filtrado['position'] = df_fifa_filtrado['position'].map(posicion_en_espanol)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6088\3608969050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fifa_filtrado['position'] = df_fifa_filtrado['position'].map(posicion_en_espanol)


: 

Se guarda los dataFrame en formato csv

In [ ]:

df_spoty_filtrado.to_csv(os.path.join(processed_path, 'spotify_filtrado_pandas.csv'), index=False, sep= ',', encoding='utf-8')
df_lagos_filtrado.to_csv(os.path.join(processed_path, 'lagos_filtrado_pandas.csv'), index=False, sep= ',', encoding='utf-8')
df_fifa_filtrado.to_csv(os.path.join(processed_path, 'fifa_filtrado_pandas.csv'), index=False, sep= ',', encoding='utf-8')


: 